In [6]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import optuna
from optuna.samplers import RandomSampler
from sklearn.model_selection import StratifiedKFold


In [3]:
num_models = 10
fold_number = 5

In [10]:
training_data = pd.read_csv('training_data.csv')
test_data = pd.read_csv('test_data.csv')
X_data = pd.concat(training_data, test_data)
y_data = pd.read_csv('y_data.csv')


In [12]:
model_preds_train = np.zeros(shape=(num_models, 188533))
model_preds_test = np.zeros(shape=(num_models, 125690))

for model_number in range(num_models):
    A = np.random.randn(X_data.shape[1], X_data.shape[1])
    Q, R = np.linalg.qr(A)
    X_data = X_data  @ Q
        
    def objective(trial):
        n_estimator = trial.suggest_int("n_estimators", 500, 1200)
        params = {
            "num_leaves" : trial.suggest_int("num_leaves", 300, 700),
            "max_depth" : trial.suggest_int("max_depth", 10, 60),
            "learning_rate" : trial.suggest_float("learning_rate", 0.001, 1.0, log=True),
            "subsample" : trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree" : trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_alpha" : trial.suggest_float("reg_alpha", 1e-7, 0.1, log=True),
            "reg_lambda" : trial.suggest_float("reg_lambda", 1e-3, 1e1, log=True),
            "min_data_in_leaf" : trial.suggest_int("min_data_in_leaf", 40, 120),
            "feature_fraction" : trial.suggest_float("feature_fraction", 0.5, 1.0),
            "data_sample_strategy" : "bagging",
            "bagging_fraction" : trial.suggest_float("bagging_fraction", 0.5, 1.0),
            "bagging_freq" : trial.suggest_int("bagging_freq", 1, 5),
            'min_child_weight': trial.suggest_float('min_child_weight', 1e-4, 1e2, log=True),
            'cat_smooth': trial.suggest_float('cat_smooth', 10, 100, log=True),
            "log_period" : 1000,
            "random_state" : 129,
            "verbose" : -1}
        train_predictions = np.zeros(188533)

        skf = StratifiedKFold(n_splits=fold_number, shuffle=True, random_state=42)

        for fold_idx, (train_indices, val_indices) in enumerate(skf.split(X_data.head(188533), y_data)):
            X_train_fold, y_train_fold = X_data.iloc[train_indices], y_data.iloc[train_indices]
            X_val_fold, y_val_fold = X_data.iloc[val_indices], y_data.iloc[val_indices]

            train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
            valid_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)

            model = lgb.train(
                params,
                train_data,
                num_boost_round=n_estimator,
                valid_sets=[valid_data],
                callbacks=[lgb.early_stopping(stopping_rounds=50)]
            )

            train_predictions[val_indices] = model.predict(X_val_fold, num_iteration=model.best_iteration)

        rmse = mean_squared_error(train_predictions, y_data) ** 0.5

        return(rmse)
    sampler = RandomSampler(seed=10) 
    study = optuna.create_study(sampler=sampler, direction='minimize')
    study.optimize(objective, n_trials = 10)
    best_trial = study.best_trial
    best_params = best_trial.params
    n_estimators = best_params['n_estimators']
    skf = StratifiedKFold(n_splits=fold_number, shuffle=True, random_state=42)
    train_predictions = np.zeros(188533)
    test_predictions = np.zeros(125690)
    for fold_idx, (train_indices, val_indices) in enumerate(skf.split(X_data.head(188533), y_data)):
        X_train_fold, y_train_fold = X_data.iloc[train_indices], y_data.iloc[train_indices]
        X_val_fold, y_val_fold = X_data.iloc[val_indices], y_data.iloc[val_indices]

        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        valid_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)

        model = lgb.train(
            best_params,
            train_data,
            num_boost_round=n_estimators,
            valid_sets=[valid_data],
            callbacks=[lgb.early_stopping(stopping_rounds=50)]
        )

        train_predictions[val_indices] = model.predict(X_val_fold, num_iteration=model.best_iteration)
        test_predictions += model.predict(X_data[188533 : (188533+125690)], num_iteration = model.best_iteration) * (1 / 5)
    
    model_preds_train[model_number] = train_predictions
    model_preds_test[model_number] = test_predictions

[I 2024-09-22 20:50:52,097] A new study created in memory with name: no-name-245defcf-17ef-4f89-a722-675d9c611c1d


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[13]	valid_0's l2: 5.38059e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[13]	valid_0's l2: 5.34071e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[15]	valid_0's l2: 5.32184e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[15]	valid_0's l2: 5.32563e+09
Training until validation scores don't improve for 50 rounds


[I 2024-09-22 20:51:05,135] Trial 0 finished with value: 73016.53477634478 and parameters: {'n_estimators': 1040, 'num_leaves': 308, 'max_depth': 42, 'learning_rate': 0.17636469336159113, 'subsample': 0.7492535061512953, 'colsample_bytree': 0.6123983227654238, 'reg_alpha': 1.54304001490974e-06, 'reg_lambda': 1.1018509458263566, 'min_data_in_leaf': 53, 'feature_fraction': 0.5441699070870052, 'bagging_fraction': 0.8426799091838986, 'bagging_freq': 5, 'min_child_weight': 0.00010560624429728648, 'cat_smooth': 32.52312463051321}. Best is trial 0 with value: 73016.53477634478.


Early stopping, best iteration is:
[15]	valid_0's l2: 5.28829e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[336]	valid_0's l2: 5.31705e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[389]	valid_0's l2: 5.26613e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[318]	valid_0's l2: 5.26612e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[368]	valid_0's l2: 5.26173e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[360]	valid_0's l2: 5.24424e+09


[I 2024-09-22 20:53:40,410] Trial 1 finished with value: 72602.02793396756 and parameters: {'n_estimators': 1069, 'num_leaves': 545, 'max_depth': 46, 'learning_rate': 0.007509797119626296, 'subsample': 0.9588870612564717, 'colsample_bytree': 0.8572878916988453, 'reg_alpha': 0.0001799973901160094, 'reg_lambda': 0.0037040785712895334, 'min_data_in_leaf': 70, 'feature_fraction': 0.8370668075331726, 'bagging_fraction': 0.720916587211498, 'bagging_freq': 3, 'min_child_weight': 0.5088641679920515, 'cat_smooth': 32.59404363006922}. Best is trial 1 with value: 72602.02793396756.


...

Early stopping, best iteration is:
[5]	valid_0's l2: 5.33026e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[224]	valid_0's l2: 5.31661e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[284]	valid_0's l2: 5.2821e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[306]	valid_0's l2: 5.26571e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[286]	valid_0's l2: 5.25921e+09
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[285]	valid_0's l2: 5.22921e+09


[I 2024-09-23 18:34:47,186] Trial 8 finished with value: 72598.68834861966 and parameters: {'n_estimators': 972, 'num_leaves': 363, 'max_depth': 12, 'learning_rate': 0.010314487673603769, 'subsample': 0.5540318863897262, 'colsample_bytree': 0.5894514042855452, 'reg_alpha': 0.02065202190439409, 'reg_lambda': 0.02893742513973448, 'min_data_in_leaf': 57, 'feature_fraction': 0.8762480851093014, 'bagging_fraction': 0.5534397921967846, 'bagging_freq': 4, 'min_child_weight': 0.06587365495478886, 'cat_smooth': 39.651139374964735}. Best is trial 8 with value: 72598.68834861966.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[603]	valid_0's l2: 5.48515e+09
Training until validation scores don't improve for 50 rounds


[W 2024-09-23 18:35:13,918] Trial 9 failed with parameters: {'n_estimators': 603, 'num_leaves': 373, 'max_depth': 42, 'learning_rate': 0.0013992136034385533, 'subsample': 0.6243062539013847, 'colsample_bytree': 0.7712042581140022, 'reg_alpha': 2.294246403767406e-06, 'reg_lambda': 0.03354641687863073, 'min_data_in_leaf': 114, 'feature_fraction': 0.9626784364338884, 'bagging_fraction': 0.7833749622874999, 'bagging_freq': 3, 'min_child_weight': 0.00012278919376690123, 'cat_smooth': 95.03843220828843} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\lerch\miniconda3\envs\cars\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\lerch\AppData\Local\Temp\ipykernel_10548\1393310619.py", line 40, in objective
    model = lgb.train(
            ^^^^^^^^^^
  File "c:\Users\lerch\miniconda3\envs\cars\Lib\site-packages\lightgbm\engine.py", li

KeyboardInterrupt: 